<a href="https://colab.research.google.com/github/umerhasan17/NLPzoo/blob/master/text_summarisation_ProphetNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ProphetNet

ProphetNet is a Seq2Seq pre-training model`. The model is optimized by **n-step ahead prediction** aiming to predict the next n tokens simultaneously. This is an alternative to the traditional **1-step** Seq2Seq model. Modelling for n-steps prevents overfitting on 'strong local correlations' by encouraging planning for future tokens.

# What are pre-trained models?

A model that is trained to solve a problem similar to the one we want to solve on a large benchmark dataset. This can take the form of labeled data or unlabeled data with 'specific self-supervised objectives'. Pre-trained models are then 'fine-tuned to adapt to downstream tasks.'

**Key terms to investigate:** Autoregressive language modelling, teacher forcing, bigram combination, greedy decoding, beam search.

# Implementation Details

**The Objective**

**N-Stream Self-Attention Mechanism**

* ProphetNet contains **n-stream self-attention** models with the main stream being the same as the self-attention in the original Transformer. 
* The parameters of the main stream are shared with every other predicting stream, this allows us to disable the n-stream during inference and only predict the next token. 
* Weights are assigned to loss values for all streams (giving higher weight to closer tokens is similar to the discount factor of future reward in reinforcement learning).

**Modified positional embedding**

**Mask based auto-encoder denoising task for Seq2Seq pre-training**
